In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import preprocessing

## Predspracovanie datasetov

Na predspracovanie oboch datasetov bolo nutné premiestniť všetky potrebné funkcie a triedy, vytvorené v predchádzajúcej
fáze projektu do novovytvoreného súboru preprocessing.py. V danom súbore sme vytvorili funkciu, ktorá ako argument berie
.csv súbory pre osobné a aj medicínske dáta, spojí ich do jedného datasetu, ktorý následne upraví pomocou našej
pipeline. Po predspracovaní nám vráti upravený dataset, s ktorým už môžeme ďalej pracovať. Túto funkciu zavoláme pre
trénovací, ako aj validačný dataset.

In [4]:
medical_train = pd.read_csv("dataset/other_train.csv")
personal_train = pd.read_csv("dataset/personal_train.csv")

train_data = preprocessing.preprocess_dataset(medical_train, personal_train)
#train_data.info()

Median stlpca education-num 10.0
Median stlpca capital-gain 0.0
Median stlpca age 52.0
Median stlpca pregnant 0.0
Median stlpca hours-per-week 40.0
Median stlpca capital-loss 0.0


d:\dokumenty\stu - fiit\5. sem\iau\projekt\venv\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
D:\Dokumenty\STU - FIIT\5. sem\IAU\projekt\preprocessing.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["pregnant"] = X["pregnant"].map(format_pregnancy)
D:\Dokumenty\STU - FIIT\5. sem\IAU\projekt\preprocessing.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [5]:
medical_valid = pd.read_csv("dataset/other_valid.csv")
personal_valid = pd.read_csv("dataset/personal_valid.csv")

valid_data = preprocessing.preprocess_dataset(medical_valid, personal_valid)
#valid_data.info()

Median stlpca education-num 10.0
Median stlpca capital-gain 0.0
Median stlpca age 52.0
Median stlpca pregnant 0.0
Median stlpca hours-per-week 40.0
Median stlpca capital-loss 0.0


d:\dokumenty\stu - fiit\5. sem\iau\projekt\venv\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
D:\Dokumenty\STU - FIIT\5. sem\IAU\projekt\preprocessing.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["pregnant"] = X["pregnant"].map(format_pregnancy)
D:\Dokumenty\STU - FIIT\5. sem\IAU\projekt\preprocessing.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i